Imports and functions

In [ ]:
import os
import netCDF4 as nc4
import numpy as np

User inputs

In [ ]:
indir = '/home/disk/bob/gpm/nam_ku/classify/ex_data/2014/06/test'
#indir = '/home/disk/bob/gpm/nam_ku/classify/ex_data/2014/07/test'
#indir = '/home/disk/bob/gpm/nam_ku/classify/ex_data/2014/10/test'
#indir = '/home/disk/bob/gpm/nam_ku/classify/ex_data/2014/11/test'
#indir = '/home/disk/bob/gpm/nam_ku/classify/ex_data/2014/12/test'
#indir = '/home/disk/bob/gpm/nam_ku/classify/ex_data/2015/02/test'
#indir = '/home/disk/bob/gpm/nam_ku/classify/ex_data/2015/04/test'
#indir = '/home/disk/bob/gpm/nam_ku/classify/ex_data/2015/06/test'
#indir = '/home/disk/bob/gpm/nam_ku/classify/ex_data/2015/07/test'
#indir = '/home/disk/bob/gpm/nam_ku/classify/ex_data/2016/01/test'
#indir = '/home/disk/bob/gpm/nam_ku/classify/ex_data/2016/04/test'
THRESHOLD = 40.0  #dBZ
MIN_HT = 3.0      #km
MAX_HT = 5.0      #km
VERT_RES = 0.125  #km

In [ ]:
#for fname in os.listdir(indir):
#   print fname

In [ ]:
#fname = 'GPM2Ku4_uw1_20140630.202105_to_20140630.202518_001916_NAM.nc'
#fname = 'GPM-2Ku.007185.20150604.111137.nc'
fname = 'GPM-2Ku.001779.20140622.005639.nc'

In [ ]:
    ncid = nc4.Dataset(indir+'/'+fname,'a')

In [ ]:
    refl = ncid.variables['refl'][:]
    (ntime,nalt,nlat,nlon) = refl.shape
    #refl_missing = ncid.variables['refl'].missing_value
    #refl[(refl==refl_missing)] = np.nan

In [ ]:
    rt = ncid.variables['rain_type'][:]
    rt_missing = ncid.variables['rain_type'].missing_value
    #rt[(rt==rt_missing)] = np.nan

In [ ]:
    rt_uw = np.array(rt, copy=True)

In [ ]:
    #rt_uw = np.ma.copy(rt)

    change rain_type to stratiform in rt_uw if:
    1. rain_type = convective and
    2. max refl in column > THRESHOLD and
    3. ht of max refl in column is between MIN_HT and MAX_HT

In [ ]:
    conv_mask = np.zeros(rt.shape,dtype=int)
    conv_mask[(rt==2)] = 1
    max_refl = np.zeros(rt.shape,dtype=float)
    max_refl = np.amax(refl,axis=1)
    max_refl_ht = np.zeros(rt.shape,dtype=float)
    max_refl_ht = np.argmax(refl,axis=1)

In [ ]:
    change = np.logical_and(np.logical_and(conv_mask==1,max_refl>=THRESHOLD),
                            np.logical_and(max_refl_ht*VERT_RES>=MIN_HT,
                                           max_refl_ht*VERT_RES<=MAX_HT))

In [ ]:
    rt_uw[change==True]=1

    Add new raintype field to input netcdf file

In [ ]:
    rt_uw_id = ncid.createVariable('rain_type_uw','f4',
                                  ('time','latitude','longitude'),zlib=True)
    #rt_uw_id = ncid.createVariable('rain_type_uw','f4',
    #                              ('time','latitude','longitude'))
    rt_uw_id.units = "none"
    rt_uw_id.long_name = "Rain Type UW"
    rt_uw_id.stratiform = 1
    rt_uw_id.convective = 2
    rt_uw_id.other = 3
    rt_uw_id.missing_value = rt_missing
    rt_uw_id[:,:,:] = rt_uw

In [ ]:
    ncid.close()